In [ ]:
! pip install basemap

import pandas as pd
import numpy as np
from scipy import stats

import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.basemap import Basemap

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split

from textblob import TextBlob
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Load the CSV file into a Pandas DataFrame
df = pd.read_csv('/kaggle/input/new-york-city-airbnb-open-data/AB_NYC_2019.csv')

# Explorative Data Analysis

First, gain an overview over all the variables in the dataset.

In [ ]:
df.describe(include='all')

Next, verify how many null values the dataset contains:

In [ ]:
df.isnull().sum()

If there are no reviews yet, the variable `last_review` and `reviews_per_month` contain null values.
The missing `host` and `host_name` values must be checked:

In [ ]:
df[df['name'].isnull()]

In [ ]:
df[df['host_name'].isnull()]

There is no obvious reason why these values are missing. Therefore remove them from the dataset:

In [ ]:
df = df.dropna(subset=['host_name', 'name'])
df.isnull().sum()

Next up get the categorical and numeric columns and check their scale. This is important for encoding later.

In [ ]:
categorical_cols = df.select_dtypes(include=['object', 'category']).columns
numeric_cols = df.select_dtypes(include=['int', 'float']).columns

In [ ]:
categorical_cols

In [ ]:
numeric_cols

Together with the `df.describe(include='all')` output above, the variable scales where identified as shown in the table below.

Nominal             | Ordinal     | Interval  | Ratio
--------------------|-------------|-----------|-------------------------------
name                | last_review | latitude  | id
host_name           |             | longitude | host_id
neighbourhood_group |             |           | price
neighbourhood       |             |           | minimum_nights
room_type           |             |           | number_of_reviews
&nbsp;              |             |           | reviews_per_month
&nbsp;              |             |           | calculated_host_listings_count
&nbsp;              |             |           | availability_365

Please note: `last_review` is ordinal because it can be compared. It will be converted to a numerical variable later.

Next up, check the distribution of all numerical variables:

In [ ]:
# initialize figure with 5 subplots in a row
fig, ax = plt.subplots(2, 5, figsize=(10, 6))

index = 0
for i, row in enumerate(ax):
    for j, cell in enumerate(row):
        if index < 0 or index >= len(numeric_cols):
            break
        
        # draw boxplot
        sns.boxplot(data=df[numeric_cols[index]], ax=ax[i][j])
        ax[i][j].set_xlabel(numeric_cols[index])
        index += 1
        cell.set_xticklabels([])

plt.subplots_adjust(left=1, bottom=0.1, right=2.5, top=2, wspace=0.5, hspace=0.4)
plt.show()

We see many outliers in the `price`, `minimum_nights`, `number_of_reviews` (and therefore `reviews_per_month`) as well as `calculated_host_listings_count`.

Lets investigate in this.

First, check if all the `host_id`s have the same `calculated_host_listings_count`. By checking this we know if the calculation of the `calculated_host_listings_count` was made when the listing was recorded.

In [ ]:
# Group by 'host_id' and calculate the maximum and minimum values of 'calculated_host_listings_count' within each group
grouped = df.groupby('host_id')['calculated_host_listings_count'].agg(['max', 'min'])

# Check if all groups have the same maximum and minimum values
all_same = grouped['max'].eq(grouped['min']).all()

if all_same:
    print("All distinct host_id values have the same calculated_host_listings_count.")
else:
    print("Distinct host_id values have different calculated_host_listings_count values.")

Since for every `host_id` the amount of `calculated_host_listings_count`s is exactly the same, we can assume that it does not matter which `calculated_host_listings_count` value is used for the next calculation.

Lets figure out where we have a "knee" in the amount of listings per host:

In [ ]:
# Group by 'host_id' and aggregate distinct values of 'calculated_host_listings_count'
unique_hosts_and_counts = df.groupby(
    'host_id')['calculated_host_listings_count'].agg(lambda x: max(x)).reset_index()

# Filter to include only rows where distinct 'calculated_host_listings_count' values are greater than 1
filtered_hosts = unique_hosts_and_counts[unique_hosts_and_counts['calculated_host_listings_count'].apply(
    lambda x: x > 1 and x <= 20)]

sorted_hosts = filtered_hosts.sort_values(
    by='calculated_host_listings_count', ascending=False)

# Calculate the knee point for the line plot
hist, bins = np.histogram(
    sorted_hosts['calculated_host_listings_count'],
    bins=range(int(min(sorted_hosts['calculated_host_listings_count'])), int(
        max(sorted_hosts['calculated_host_listings_count'])) + 1)
)

# Create a line plot
plt.figure(figsize=(8, 6))
plt.plot(bins[:-1], hist, marker='o', linestyle='-')
plt.xlabel('calculated_host_listings_count')
plt.ylabel('Frequency')
plt.title('Line Plot to find Knee Point')
plt.grid(True)
plt.show()

# Filter to include only rows where distinct 'calculated_host_listings_count' values are greater than 1
filtered_hosts = unique_hosts_and_counts[unique_hosts_and_counts['calculated_host_listings_count'].apply(
    lambda x: x >= 5)]
sorted_hosts = filtered_hosts.sort_values(
    by='calculated_host_listings_count', ascending=False)

pd.set_option('display.max_rows', 50)
sorted_hosts

The knee appears at 5 listings. Therefore mark all hosts with 5 or more listings as `is_professional`.

Things that were observed during the descriptive analysis:
- There is a host "Sonder (NYC)" with 96 listings and a host "Sonder" with 327 listings. Depending on the analysis, the data of the two hosts should be merged.
- Other options to figure out if a host is professional (i.e. a company) failed. We tried "nltk (Natural Language Toolkit)" and even manual methods.

Next, check if the `host_name` is unique:

In [ ]:
# Group the DataFrame by 'host_name' and get unique 'calculated_host_listings_count' values within each group
grouped = df.groupby('host_name')['calculated_host_listings_count'].unique()

# Filter groups where there is more than one unique value for 'calculated_host_listings_count'
different_counts = grouped[grouped.apply(lambda x: len(x) > 1)]

# Print the 'host_name' values with different 'calculated_host_listings_count'
print(different_counts.index.tolist())

The host_name is not unique. Therefore stick with the `host_id` for the grouping to calculate the `is_professional` variable.

In [ ]:
def is_professional(count):
    return 1 if count >= 5 else 0

# Apply the function to create the 'is_professional' variable
df['is_professional'] = df['calculated_host_listings_count'].apply(is_professional)

Now check how many listings are most likely from a professional real estate owner:

In [ ]:
# Get the value counts for 'is_professional' column
value_counts = df['is_professional'].value_counts()

# Calculate relative values as percentages
relative_values = (value_counts / value_counts.sum()) * 100

print('Absolute numbers:')
print(value_counts)
print('')
print('Relative numbers:')
print(relative_values)

Lets try to find out more about professional listings by plotting their location, and compare by neighborhood.

In [ ]:
prof_df = df[df['is_professional'] == 1]
m = Basemap(
    llcrnrlon=prof_df['longitude'].min() - 0.05, 
    llcrnrlat=prof_df['latitude'].min() - 0.05,
    urcrnrlon=prof_df['longitude'].max() + 0.05,
    urcrnrlat=prof_df['latitude'].max() + 0.05, 
    resolution='i'
)

# m.bluemarble(scale=5.0)
m.drawcoastlines()
m.drawcountries(linewidth=1, linestyle='solid', color='k' )

x, y = m(prof_df['longitude'].values, prof_df['latitude'].values)
m.scatter(x, y, c='r', marker='o', s=3, label=f'Professional listings')

plt.legend()
plt.show()

Professional listings tend to cluster around the Manhatten area.

This gets confirmed by looking at the numbers:

In [ ]:
# Group the DataFrame by neighborhood and room_type and count the occurrences
grouped = df.groupby(['neighbourhood_group', 'is_professional']).size().unstack(fill_value=0)

# Create a stacked bar chart
ax = grouped.plot(kind='bar', stacked=True, figsize=(10, 6))

# Customize the plot
plt.title('Professional Listening counts by Neighborhood')
plt.xlabel('Neighborhood')
plt.ylabel('Count')
plt.xticks(rotation=0)  # Rotate x-axis labels if needed

# Display the legend
plt.legend(title='Is Professional')

# Show the plot
plt.show()

Next, lets compare the relative amount of listings by room type and district (neighbourhood group).

In [ ]:
# Group the DataFrame by room_type and count the occurrences
room_type_counts = df['room_type'].value_counts()

# Create a pie chart
plt.figure(figsize=(8, 8))
plt.pie(room_type_counts, labels=room_type_counts.index, autopct='%1.1f%%', startangle=140)
plt.title('Relative Amount of Room Types')
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

# Display the pie chart
plt.show()

# Group the DataFrame by room_type and count the occurrences
room_type_counts = df['neighbourhood_group'].value_counts()

# Create a pie chart
plt.figure(figsize=(8, 8))
plt.pie(room_type_counts, labels=room_type_counts.index, autopct='%1.1f%%', startangle=140)
plt.title('Relative Amount of listings by neighbourhood group')
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

# Display the pie chart
plt.show()

As the graphs show, there are almost no shared rooms and the most popular districts are Manhattan and Brooklyn. Based on internet research, each district has its own distinct characteristics:

1. Manhattan:
    - Manhattan is known as the heart of New York City and is often associated with iconic landmarks like Times Square, Central Park, and the Empire State Building.
    - It's the financial and commercial hub of the city, home to Wall Street and numerous corporate headquarters.
    - Manhattan offers a vibrant nightlife, world-class dining, and a diverse cultural scene with theaters, museums, and art galleries.

1. Brooklyn:
    - Brooklyn is known for its distinct neighborhoods, each with its own character and charm.
    - It's known for its creative and artistic communities, with areas like Williamsburg and DUMBO being hubs for artists and musicians.
    - Brooklyn offers beautiful parks, historic brownstone-lined streets, and scenic waterfront views.
    - The borough has a strong sense of community and is known for its cultural diversity.

1. Staten Island:
    - Staten Island is often considered the most suburban of the five boroughs, with a more relaxed pace of life.
    - It's known for its natural beauty, including parks, forests, and waterfront areas.
    - The Staten Island Ferry provides stunning views of the Statue of Liberty and the Manhattan skyline.
    - Staten Island has a strong sense of community and local pride.

1. Bronx:
    - The Bronx is known for its rich cultural heritage, including a strong presence of Latinx and African American communities.
    - It's home to the Bronx Zoo, Yankee Stadium, and the birthplace of hip-hop.
    - The Bronx offers a mix of residential neighborhoods and green spaces, such as the Bronx Botanical Garden.

1. Queens:
    - Queens is one of the most diverse places in the world, with residents from all over the globe.
    - It's known for its cultural festivals, ethnic cuisine, and vibrant street life.
    - Queens has a variety of residential neighborhoods, parks, and cultural institutions.
    - The borough is often celebrated for its sense of inclusion and diversity.


Given this, most AirBnB listings are unsurprisingly located in the top tourist destinations.


When comparing the room type with the neighbourhood, no abnormalities can be detected:

In [ ]:
# Group the DataFrame by neighborhood and room_type and count the occurrences
grouped = df.groupby(['neighbourhood_group', 'room_type']).size().unstack(fill_value=0)

# Create a stacked bar chart
ax = grouped.plot(kind='bar', stacked=True, figsize=(10, 6))

# Customize the plot
plt.title('Room Type Counts by Neighbourhood')
plt.xlabel('Neighborhood')
plt.ylabel('Count')
plt.xticks(rotation=0)  # Rotate x-axis labels if needed

# Display the legend
plt.legend(title='Room Type')

# Show the plot
plt.show()

The next visualisations aim to shed light on the variables `price`, `minimum_nights`, `number_of_reviews`, `reviews_per_month`, `calculated_host_listings_count` and `availability_365`:

In [ ]:
# Create a pivot table or crosstab
heatmap = pd.crosstab(index=df['room_type'], columns=df['neighbourhood_group'], values=df['price'], aggfunc='mean')

# Plot the confusion matrix as a heatmap
plt.figure(figsize=(10, 6))
plt.imshow(heatmap, cmap='Blues', interpolation='nearest')
plt.title('Heatmap - Average price')
plt.xlabel('Neighborhood Group')
plt.ylabel('Room Type')
plt.colorbar()

# Display the values in the cells
for i in range(len(heatmap.index)):
    for j in range(len(heatmap.columns)):
        plt.text(j, i, f'{heatmap.iloc[i, j]:.2f}', ha='center', va='center', fontsize=12)

plt.xticks(range(len(heatmap.columns)), heatmap.columns)
plt.yticks(range(len(heatmap.index)), heatmap.index)
plt.tight_layout()

# Show the plot
plt.show()

The highest average price is found for entire homes in Manhattan. Please note: The results are the same if the median is used in order to be more safe in regards to outliers.

In [ ]:
# Create a pivot table or crosstab
heatmap = pd.crosstab(index=df['room_type'], columns=df['neighbourhood_group'], values=df['minimum_nights'], aggfunc='mean')

# Plot the confusion matrix as a heatmap
plt.figure(figsize=(10, 6))
plt.imshow(heatmap, cmap='Blues', interpolation='nearest')
plt.title('Heatmap - Average minimum nights')
plt.xlabel('Neighborhood Group')
plt.ylabel('Room Type')
plt.colorbar()

# Display the values in the cells
for i in range(len(heatmap.index)):
    for j in range(len(heatmap.columns)):
        plt.text(j, i, f'{heatmap.iloc[i, j]:.2f}', ha='center', va='center', fontsize=12)

plt.xticks(range(len(heatmap.columns)), heatmap.columns)
plt.yticks(range(len(heatmap.index)), heatmap.index)
plt.tight_layout()

# Show the plot
plt.show()

Similarly to the price, the average amount of minimum nights is the highest for entire homes in Manhatten.
Please note: The values are distorted by outliers. Compare with the median values which are equal in Brooklyn and Manhattan.

In [ ]:
# Create a pivot table or crosstab
heatmap = pd.crosstab(index=df['room_type'], columns=df['neighbourhood_group'], values=df['number_of_reviews'], aggfunc='mean')

# Plot the confusion matrix as a heatmap
plt.figure(figsize=(10, 6))
plt.imshow(heatmap, cmap='Blues', interpolation='nearest')
plt.title('Heatmap - Average number of reviews')
plt.xlabel('Neighborhood Group')
plt.ylabel('Room Type')
plt.colorbar()

# Display the values in the cells
for i in range(len(heatmap.index)):
    for j in range(len(heatmap.columns)):
        plt.text(j, i, f'{heatmap.iloc[i, j]:.2f}', ha='center', va='center', fontsize=12)

plt.xticks(range(len(heatmap.columns)), heatmap.columns)
plt.yticks(range(len(heatmap.index)), heatmap.index)
plt.tight_layout()

# Show the plot
plt.show()

Checking the average number of reviews shows that listings outside the touristic areas get approximately one third more reviews.

In [ ]:
# Create a pivot table or crosstab
heatmap = pd.crosstab(index=df['room_type'], columns=df['neighbourhood_group'], values=df['reviews_per_month'], aggfunc='mean')

# Plot the confusion matrix as a heatmap
plt.figure(figsize=(10, 6))
plt.imshow(heatmap, cmap='Blues', interpolation='nearest')
plt.title('Heatmap - Average reviews per month')
plt.xlabel('Neighborhood Group')
plt.ylabel('Room Type')
plt.colorbar()

# Display the values in the cells
for i in range(len(heatmap.index)):
    for j in range(len(heatmap.columns)):
        plt.text(j, i, f'{heatmap.iloc[i, j]:.2f}', ha='center', va='center', fontsize=12)

plt.xticks(range(len(heatmap.columns)), heatmap.columns)
plt.yticks(range(len(heatmap.index)), heatmap.index)
plt.tight_layout()

# Show the plot
plt.show()

Unsurprisingly, this is the same for average reviews per month as these values correlate.

In [ ]:
# Create a pivot table or crosstab
heatmap = pd.crosstab(index=df['room_type'], columns=df['neighbourhood_group'], values=df['calculated_host_listings_count'], aggfunc='mean')

# Plot the confusion matrix as a heatmap
plt.figure(figsize=(10, 6))
plt.imshow(heatmap, cmap='Blues', interpolation='nearest')
plt.title('Heatmap - Average calculated host listings count')
plt.xlabel('Neighborhood Group')
plt.ylabel('Room Type')
plt.colorbar()

# Display the values in the cells
for i in range(len(heatmap.index)):
    for j in range(len(heatmap.columns)):
        plt.text(j, i, f'{heatmap.iloc[i, j]:.2f}', ha='center', va='center', fontsize=12)

plt.xticks(range(len(heatmap.columns)), heatmap.columns)
plt.yticks(range(len(heatmap.index)), heatmap.index)
plt.tight_layout()

# Show the plot
plt.show()

Since we already showed that the highest amount of listing from professionals is focused in the touristic areas, it is no surprise that average calculated host listings count peaks for Manhattan.

More surprising is the focus on entire homes / apartments.

In [ ]:
# Create a pivot table or crosstab
heatmap = pd.crosstab(index=df['room_type'], columns=df['neighbourhood_group'], values=df['availability_365'], aggfunc='mean')

# Plot the confusion matrix as a heatmap
plt.figure(figsize=(10, 6))
plt.imshow(heatmap, cmap='Blues', interpolation='nearest')
plt.title('Heatmap - Average availability 365')
plt.xlabel('Neighborhood Group')
plt.ylabel('Room Type')
plt.colorbar()

# Display the values in the cells
for i in range(len(heatmap.index)):
    for j in range(len(heatmap.columns)):
        plt.text(j, i, f'{heatmap.iloc[i, j]:.2f}', ha='center', va='center', fontsize=12)

plt.xticks(range(len(heatmap.columns)), heatmap.columns)
plt.yticks(range(len(heatmap.index)), heatmap.index)
plt.tight_layout()

# Show the plot
plt.show()

Comparing the number of days when the listing is available for booking, we see that the lowest values can be found for shared rooms in staten island and the highest value for private rooms in Staten Island.

The same applies when checking the median. However, comparing the median, the values for Manhattan come down, which could be explained by having a few expensive and extravagant listings which stay empty for most of the time.

In order to get a more unbiased picture of the price, calculate a price scale based on the mean price per neighbourhood:

In [ ]:
df_price = df.groupby(['neighbourhood_group', 'neighbourhood'])['price'].mean().reset_index()
df_price['price'].plot(kind='hist')

In [ ]:
df_price['price'].describe()

Use pandas qcut to find equal-sized buckets:

In [ ]:
pd.qcut(df_price['price'], q=4)

Based on these bins, create a pricing category:

In [ ]:
price_bins = [0, 81.731, 101.8, 152.714, float('inf')]  # Adjust the price thresholds as needed
price_labels = [0, 1, 2, 3]

In [ ]:
# Create a new column 'price_category' based on price
df['price_category'] = pd.cut(df['price'], bins=price_bins, labels=price_labels, right=False)

In [ ]:
df['price_category'].value_counts()

Lets check if there are pricing differences in the categories used by individuals and "companies":

In [ ]:
# Create a pivot table or crosstab
# Create a pivot table with counts
heatmap = pd.crosstab(index=df['is_professional'], columns=df['price_category'], values=df['calculated_host_listings_count'], aggfunc='count')

# Calculate relative counts as percentages
total_counts = heatmap.sum().sum()  # Calculate the total count
relative_heatmap = (heatmap / total_counts * 100).astype(int).astype(str) + '%'  # Calculate percentages and format as strings

# Plot the confusion matrix as a heatmap
plt.figure(figsize=(10, 6))
plt.imshow(heatmap, cmap='Blues', interpolation='nearest')
plt.title('Heatmap - Price category of professional offerings')
plt.xlabel('Price Category')
plt.ylabel('Is professional')
plt.colorbar()

# Display the values in the cells
for i in range(len(heatmap.index)):
    for j in range(len(heatmap.columns)):
        plt.text(j, i, f'{relative_heatmap.iloc[i, j]}', ha='center', va='center', fontsize=12)

plt.xticks(range(len(heatmap.columns)), heatmap.columns)
plt.yticks(range(len(heatmap.index)), heatmap.index)
plt.tight_layout()

# Show the plot
plt.show()

The result is bimodal, but the degree is much greater with professional hosts (category 0 and 3 are 4 times more frequent than 1 and 2).

# Correlation

Lets calculate a correlation matrix for all numerical variables.
Since the data is not linear, use Spearman's Rank Correlation: 

In [ ]:
numeric_df = df[[*numeric_cols]]
correlation_matrix = numeric_df.corr(method='spearman')

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title("Correlation Matrix")
plt.show()

We can observe that:
- There is a correlation between location (longitude) and price
- As expected, there is a strong correlation between number of reviews and reviews per month
- We also see a correlation between id and review per month. If the ID is auto-incremental (lower IDs mean older listings), its possible that older listings have more reviews, i.e negative correlation. The same applies for the host_id. **This is important as the IDs provide hints about how long a listing / host is on the platform relative to others**
- It does not look like there is a causal relationship between `availability_365` and `calculated_host_listings_count` (see scatter plot below)
- Listings less used (higher `availability_365` values) have a higher `number_of_reviews` value. There is either no causal relationship, or AirBnB has a problem with "fake reviews".
- Companies (hosts with more listings) tent to get slightly more reviews.
- The minimum amount of nights is correlated with the number of reviews. Higher amount of minimum nights means less reviews.

In [ ]:
plt.figure(figsize=(10, 6))  # Set the figure size
plt.scatter(df['availability_365'], df['calculated_host_listings_count'], alpha=0.5)  # Create the scatter plot
plt.title('Scatter Plot of Availability vs. Host Listings Count')  # Set the title
plt.xlabel('Availability (in days)')  # Set the x-axis label
plt.ylabel('Host Listings Count')  # Set the y-axis label
plt.grid(True)  # Add a grid for reference
plt.show()  # Show the plot

The scatter plot above is only used to verify the correlation between `calculated_host_listings_count` and `availability_365` visually.


Next, check if there is a difference in correlation for extremely expensive listings.

In [ ]:
numeric_df = df[[*numeric_cols]]
correlation_matrix = numeric_df[numeric_df['price'] > 5000].corr(method='spearman')

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title("Correlation Matrix")
plt.show()

Next, lets use the Chi-squared test to check if there is a correlation between the categorical variables. For this, use the `price_category` variable created earlier.

In [ ]:
sampled_data = df.sample(frac=0.1, random_state=42)

for col in categorical_cols:
    contingency_table = pd.crosstab(sampled_data[col], sampled_data['price_category'])
    chi2, p_value, _, _ = stats.chi2_contingency(contingency_table)
    print(f"Chi-square test for column '{col}' against 'price_category':")
    print(f"Chi-square statistic: {chi2}")
    print(f"p-value: {p_value}")
    print()

All variables showed a p-value lower than 0.05. One explanation is, that larger sample sizes tend to produce smaller p-values. Therefore a random sample of 10% of the values is created. This produces the following result:
- There is no significant correlation between the `name` and the `price_category`. Either there is no correlation or there are simply too many groups.
- There is a significant correlation between the `host_name` and the `price_category`.
- The `room_type` and location (`neighbourhood_group` and `neighbourhood`) are significantly correlated with the price category.

# Feature Encoding

Most Machine Learning Algorithms are working well with non-numeric, categorical variables.

Therefore the variables must get encoded.

First, copy the dataframe to have access to the unprocessed data.

In [ ]:
encoded_df = df.copy()
# encoded_df = encoded_df.drop(['id', 'host_id'], axis=1)

Next, use sentiment analysis to assign a value between `-1` (negative sentiment) and `+1` (positive sentiment).

In [ ]:
def analyze_sentiment(text):
    if isinstance(text, str):
        analysis = TextBlob(text)
        return analysis.sentiment.polarity
    
    return float("nan")
    


encoded_df['name'] = encoded_df['name'].apply(analyze_sentiment)


Next, encode the `last_review` as timestamp counting the seconds since January 1, 1970.

In [ ]:
encoded_df['last_review'].fillna('1970-01-01', inplace=True)
encoded_df['last_review'] = pd.to_datetime(encoded_df['last_review'])
encoded_df['last_review'] = (encoded_df['last_review'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')

The variables `neighbourhood_group`, `neighbourhood` and `host_name` get encoded with the distinct value counts (which will make `host_name` correlation strongly with `calculated_host_listings_count` and therefore most likely not needed).

For `room_type`, use one hot encoding, but re-add the room_type as its helpful for visualization.

In [ ]:
encoded_df['neighbourhood_group'] = encoded_df['neighbourhood_group'].map(encoded_df['neighbourhood_group'].value_counts())
encoded_df['neighbourhood'] = encoded_df['neighbourhood'].map(encoded_df['neighbourhood'].value_counts())
encoded_df['host_name'] = encoded_df['host_name'].map(encoded_df['host_name'].value_counts())

encoded_df = pd.get_dummies(encoded_df, columns=['room_type'], prefix=['room_type'])
encoded_df = encoded_df.drop(['room_type_Shared room'], axis=1)

encoded_df['room_type'] = df['room_type']
encoded_df['neighbourhood_group_org'] = df['neighbourhood_group']


The remaining missing values (review based) get filled with 0.

In [ ]:
encoded_df.fillna(0, inplace=True)
encoded_df.isnull().sum()

In [ ]:
encoded_df.head()

# Feature Importance

Besides correlation checks, feature importance using Machine Learning Algorithms refers to the process of determining the significance or contribution of each feature (or variable) in a dataset toward predicting the target variable.

In [ ]:
X = encoded_df.drop("price", axis=1)
X = X.drop(['price_category', 'room_type', 'neighbourhood_group_org'], axis=1)
y = encoded_df["price"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
# Instantiate the Random Forest classifier
rf = RandomForestRegressor()
# Fit the model on the data
rf.fit(X_train, y_train)

# Extract feature importances
importances_rf = rf.feature_importances_


# Create a DataFrame to hold the feature importances
feature_importances = pd.DataFrame({"Feature": X.columns, "Random Forest": importances_rf})

# Sort the DataFrame by feature importance in descending order
feature_importances = feature_importances.sort_values(by="Random Forest", ascending=False)

feature_importances

Unsurprisingly, the location is the top feature to predict the price. More surprisingly, the `id` and `host_id` are among the top features to predict the price.
Based on previous finding, the RandomForestRegressor identifies the location and how long the listing is available as the top features.

In [ ]:
# Instantiate the Gradient Boosting classifier
gb = GradientBoostingRegressor()

# Fit the model on the data
gb.fit(X_train, y_train)

# Extract feature importances
importances_gb = gb.feature_importances_

feature_importances = pd.DataFrame({"Feature": X.columns, "Gradient Boosting": importances_gb})

# Sort the DataFrame by feature importance in descending order
feature_importances = feature_importances.sort_values(by="Gradient Boosting", ascending=False)

feature_importances

Things look different when using Gradient Boosting: The room type, the location and the availability are the top features to predict the price. The `id` (how long the listing is on the platform) is also responsible for 10 % of the price.


The information gained from the feature importance check will no be used for clustering.

# Price prediction

Lets try and predict the price using quantile regression with Gradient Boosting. 

In [ ]:
# Set lower and upper quantile
LOWER_ALPHA = 0.2
UPPER_ALPHA = 0.8

# Each model has to be separate
lower_model = GradientBoostingRegressor(loss="quantile", alpha=LOWER_ALPHA)
upper_model = GradientBoostingRegressor(loss="quantile", alpha=UPPER_ALPHA)

# Fit models
lower_model.fit(X_train, y_train)
upper_model.fit(X_train, y_train)

# Record actual values on the test set
predictions = pd.DataFrame(y_test)

# Predict
predictions['lower'] = lower_model.predict(X_test)
predictions['upper'] = upper_model.predict(X_test)

# Define the quantile intervals
lower_quantile = predictions['lower']
upper_quantile = predictions['upper']

# Add a new column 'price_interval' to indicate where the actual price falls
predictions['price_interval'] = 'Within Interval'
predictions.loc[y_test < lower_quantile, 'price_interval'] = 'Lower Quantile'
predictions.loc[y_test > upper_quantile, 'price_interval'] = 'Upper Quantile'

predictions

In [ ]:
predictions['price_interval'].value_counts()

The accuracy of the prediction is about 60 %. Therefore it is unlikely to achieve a significantly higher level of performance through feature engineering or hyperparameter tuning. Instead, investigation in collecting more and different variables should be made.  

# Clustering

The goal of clustering in this project is to find patterns and / or anomalies.

In order to find the ideal amount of clusters, the elbow-method is used:

In [ ]:
# Define the range of cluster numbers to test
cluster_range = range(1, 11)
inertia = []

# Calculate inertia (within-cluster sum of squares) for each cluster number
# neighbourhood_group, neighbourhood, last_review
for num_clusters in cluster_range:
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    kmeans.fit(encoded_df[
        # [
        #     'latitude', 'longitude', 'price'
        # ]
        [
            'name', 'host_name', 
            'latitude', 'longitude', 'price', 'minimum_nights', 
            'number_of_reviews', 'reviews_per_month', 'calculated_host_listings_count', 
            'availability_365', 'room_type_Entire home/apt', 'room_type_Private room', 
            'price_category'
        ]
    ])
    inertia.append(kmeans.inertia_)

# Plot the elbow curve
plt.figure(figsize=(8, 5))
plt.plot(cluster_range, inertia, marker='o', linestyle='-', color='b')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia (Within-Cluster Sum of Squares)')
plt.title('Elbow Method for Optimal Number of Clusters')
plt.grid(True)
plt.show()



Adding `id` and `host_id` causes the number of clusters to drop to 2. All other variables as shown above provide a steady amount of four clusters.


Next up, define the code to print the clusters and perform a multivariant linear regression to find the importance of each feature on the price. We played around with different models like Ordinary Least Squares (OLS) or Generalized Linear Models (GLMs) with different distributions like Gaussian, Poisson, or Gamma. Based on the price distribution (right-skewed), the Gamma distribution is preferred.

In [ ]:

def create_and_plot_cluster(columns):

    # Standardize the data (important for K-Means)
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(encoded_df[columns])
    
    # Initialize the K-Means model
    kmeans = KMeans(n_clusters=4, random_state=42)

    # Fit the model to the scaled data
    kmeans.fit(data_scaled)

    # Add cluster labels to the DataFrame
    encoded_df['cluster'] = kmeans.labels_
    
    fig = plt.figure(figsize = (12,8))

    m = Basemap(
        llcrnrlon=encoded_df['longitude'].min() - 0.05, 
        llcrnrlat=encoded_df['latitude'].min() - 0.05,
        urcrnrlon=encoded_df['longitude'].max() + 0.05,
        urcrnrlat=encoded_df['latitude'].max() + 0.05, 
        resolution='i'
    )

    # m.bluemarble(scale=5.0)
    m.drawcoastlines()
    m.drawcountries(linewidth=1, linestyle='solid', color='k' )

    # Plot points with colors based on the 'Cluster' column
    cluster_colors = ['r', 'g', 'b', 'y']  # Define colors for each cluster
    for cluster_label in encoded_df['cluster'].unique():
        cluster_data = encoded_df[encoded_df['cluster'] == cluster_label]
        x, y = m(cluster_data['longitude'].values, cluster_data['latitude'].values)
        m.scatter(x, y, c=cluster_colors[cluster_label], marker='o', s=5, label=f'Cluster {cluster_label}')

    plt.legend()
    plt.show()

    clustered_data = encoded_df.groupby('cluster')
    for cluster_label, cluster_data in clustered_data:
        cluster_centroid = kmeans.cluster_centers_[cluster_label]
        original_centroid = scaler.inverse_transform([cluster_centroid])

        print("Centroid for Cluster " + str(cluster_label) + ":")

        for i in range(len(columns)):
            print(" Feature <" + str(columns[i]) + ">: " + str(original_centroid[0][i]))

def get_multivariant_linear_regression_model(cluster_data):
    # return smf.ols(formula='price ~ name + minimum_nights + C(neighbourhood_group_org) + number_of_reviews + C(room_type) + C(is_professional)', data=cluster_data).fit()
    return smf.glm(formula='price ~ name + minimum_nights + C(neighbourhood_group_org) + number_of_reviews + C(room_type) + C(is_professional)', data=cluster_data, family=sm.families.Gamma()).fit()

def format_float(value):
    return '%.6f' % value

Lets start by clustering the location (coordinates and districts) and the price:

In [ ]:
create_and_plot_cluster(['latitude', 'longitude', 'price'])
# create_and_plot_cluster(['neighbourhood_group', 'price'])

# Perform Multivariate Linear Regression for each cluster
for cluster_label in encoded_df['cluster'].unique():
    print("")
    print("----------------------")
    print("Details for Cluster " + str(cluster_label))
    print("----------------------")
    
    pd.set_option('display.float_format', format_float)
    
    cluster_data = encoded_df[encoded_df['cluster'] == cluster_label]
    # Calculate and print the mean of all columns
    mean_values = cluster_data.select_dtypes(include=['int', 'float']).mean()
    print("Mean of all columns:")
    print(mean_values)
    
    # Print the regression summary for each cluster
    print(f"Regression Summary for Cluster {cluster_label}:")
    model = get_multivariant_linear_regression_model(cluster_data)
    print(model.summary())
    print("\n")

Unsurprisingly, the clusters (regardless of if the coordinates are being used or the districts) form around the location. The price has no real effect on the clusters as the price is correlated with the location.

Regarding the multivariant linear regression analysis:
- Cluster 1 has the highest R-squared value of 0.259. This means that only 25.9 % of the variance can be explained by the model at best.
- We see that the location would increase the price for the listing. The listings in cluster 1 (Queens, Bronx and Staten Island) can benefit from a more positive name.

Next, add the room Type to the equation.

In [ ]:
# create_and_plot_cluster(['latitude', 'longitude', 'price', 'room_type_Entire home/apt', 'room_type_Private room'])
create_and_plot_cluster(['neighbourhood_group', 'price', 'room_type_Entire home/apt', 'room_type_Private room'])

# Perform Multivariate Linear Regression for each cluster
for cluster_label in encoded_df['cluster'].unique():
    print("")
    print("----------------------")
    print("Details for Cluster " + str(cluster_label))
    print("----------------------")
    
    pd.set_option('display.float_format', format_float)
    
    cluster_data = encoded_df[encoded_df['cluster'] == cluster_label]
    # Calculate and print the mean of all columns
    mean_values = cluster_data.select_dtypes(include=['int', 'float']).mean()
    print("Mean of all columns:")
    print(mean_values)
    
    # Print the regression summary for each cluster
    print(f"Regression Summary for Cluster {cluster_label}:")
    model = get_multivariant_linear_regression_model(cluster_data)
    print(model.summary())
    print("\n")

Of course, the type of the room has a direct influence on the price. An entire house is more expensive than a shared room. However, since all room types are available in all regions, the mean price in the clusters gets reduced and geographically spoken, the cluster borders get blurred.

Next, lets find the features that provide distinct pricing groups, but without using location or pricing data:

In [ ]:
# create_and_plot_cluster(['latitude', 'longitude', 'price', 'room_type_Entire home/apt', 'room_type_Private room', 'number_of_reviews'])
create_and_plot_cluster(['name', 'calculated_host_listings_count', 'room_type_Entire home/apt', 'reviews_per_month'])
# Perform Multivariate Linear Regression for each cluster
for cluster_label in encoded_df['cluster'].unique():
    print("")
    print("----------------------")
    print("Details for Cluster " + str(cluster_label))
    print("----------------------")
    
    pd.set_option('display.float_format', format_float)
    
    cluster_data = encoded_df[encoded_df['cluster'] == cluster_label]

    print(cluster_data[['price']].mean())
    # Calculate and print the mean of all columns
    mean_values = cluster_data.select_dtypes(include=['int', 'float']).mean()
    print("Mean of all columns:")
    print(mean_values)
    
    # Print the regression summary for each cluster
    print(f"Regression Summary for Cluster {cluster_label}:")
    model = get_multivariant_linear_regression_model(cluster_data)
    print(model.summary())
    print("\n")

Surprisingly, the sentiment of the listing name can be used as a good indicator to predict the location. The more positive the listing name, the higher the chance that the location is either Manhattan or Brooklyn. Next, `calculated_host_listings_count` is a good indicator as well, as most likely professional investors tend to buy properties at prestiges locations (which are less affordable for individuals). The type of the room as well as the reviews per month help to get distinct clusters in terms of the price.

# Discussion

The variables in the Data Set are all correlated. Listings at a "better" location have higher demand, are more expensive, less available and have more reviews. However, highly expensive listings for example tend to have less reviews, and are more available.

Multivariant Regression in combination with clustering to figure out the effect of an independent variable on a dependent variable is a good way for providing information on how certain clusters may improve (in this case the price). However with this dataset, only a fraction of the variance can be explained using linear regression. From the correlation analysis we know that non-linear models deliver better result, however, using GLM with the preferred Gamma distribution provides very low coefficients. Using a Gaussian Distribution provides sometimes very unlikely results.

In order to make this approach useful, the price must be predictable first. When trying to to this using Regression, the overall accuracy is very low (around 60%). One explanation for this is the very low dimensionality of the data. For example there is no information about the size, equipment, age, number of beds, specialities, construction etc. Once the price can be predicted with higher accuracy, multivariant models can be used to predict the effect on the price fpr specific variables, in specific clusters. This will provide a huge benefit for hosts because they get recommendations on how to get more revenue from their listings and AirBnB will get more commissions.

Based on the current dataset, the following observations where made:
- Location and Room Type influences the price the most
- "Professional" Hosts tend to address the lowest and highest price segment, less in between.
- Reviews help for bookings, except for the most expensive (luxury?) listings.
